# Exercise 1:
We will write a function that:
 - Receives the path of a fasta file with the sequences we want to align
 - Returns a dictionary where keys are the name of the sequence and the value is the sequence


In [2]:
def readFastaFile(file):
    return

# Exercise 2:

We will write a function that:
 - Receives the path of a fasta file with the alignment of unbiased sequences
 - Returns a 2D dictionary where keys are the name of the residues and the value is the score


In [3]:
def generateMatrix(alignment):
    return 

# Exercise 3:
We will write a function that:
 - Receives the path of a matrix (.mat) file with a substitution matrix
 - Returns a 2D dictionary where keys are the name of the residues and the value is the score


In [4]:
def readMatrix(filename):
    
    # Read the file
    handle = open(filename, 'r')
    content= handle.readlines()
    handle.close()
    
    # Set up the matrix file
    matrix  = {}
    letters = []
    numline = len(content) 
    
    for nl in range(XXX, XXX):
        line = content[nl]
        splt = line.split()
        a = splt[XXX]
        if a not in matrix:
            matrix[a] = {}
            letters.append(a)
            
    # Go throug the file and save the values
    for nl in range(XXX, XXX):
        line = content[nl]
        splt = line.split()
        l = len(splt)
        aa1 = XXX
        for a in range(XXX, XXX):
            aa2 = letters[XXX]
            matrix[XXX][XXX] = splt[XXX]
            matrix[XXX][XXX] = XXX
    
    return matrix

# Exercise 4:
We will write a function that:
 - Receives two sequences
 - Returns both sequences aligned

# Exercise 5:
We will write a function that:
 - Receives the path of netwick file 
 - Returns an instance of class tree

# Other interresting functions already implemented